# Fashion-MNIST Classification Challenge

**Цель:** построить CNN-модель, которая на Public Leaderboard набирает `accuracy ≥ 0.93`.

Мы поэтапно:
1. Загрузим и посмотрим данные  
2. Проведём предобработку и EDA  
3. Определим архитектуру сети  
4. Настроим аугментацию и колбэки  
5. Обучим модель и сформируем сабмишен  


In [ ]:
# 1. Импорты и вывод доступных файлов
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

from sklearn.model_selection import train_test_split


In [ ]:
# 2. Загрузка датасетов
train = pd.read_csv('/kaggle/input/mipt-bio-2025/fashion-mnist-testn/fmnist_train.csv')
test  = pd.read_csv('/kaggle/input/mipt-bio-2025/fashion-mnist-testn/fmnist_test.csv')
sample_sub = pd.read_csv('/kaggle/input/mipt-bio-2025/fashion-mnist-testn/sample_submission.csv')

print("Train:", train.shape)
print("Test: ", test.shape)
print("Sample submission:", sample_sub.shape)


In [ ]:
# 3. Убираем пропуски (если есть) и отделяем X/y
train_clean = train.dropna(axis=0)

X = train_clean.drop(['label', 'Id'], axis=1).values
y = train_clean['label'].values
X_test = test.drop('Id', axis=1).values

# Нормализация: 0–1
X = X.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Reshape для CNN
X = X.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# One-hot кодирование меток
y_cat = to_categorical(y, num_classes=10)

In [ ]:
# 4. Визуализируем по 10 первых картинок
fig, axes = plt.subplots(2, 5, figsize=(12,5))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(X[i].squeeze(), cmap='gray')
    ax.set_title(f'Label: {y[i]}')
    ax.axis('off')
plt.tight_layout()


### 5. Создаём функцию для модели
- Две свёрточные группы с BatchNorm + Dropout  
- Полносвязный слой 256 + Dropout  
- Выход softmax на 10 классов  

In [ ]:
def make_model():
    inp = Input(shape=(28,28,1))
    # Блок 1
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.25)(x)

    # Блок 2
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.25)(x)

    # Полносвязная часть
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    out = layers.Dense(10, activation='softmax')(x)
    model = models.Model(inp, out)
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

model = make_model()
model.summary()

In [ ]:
# 6. Разбиение на train/val
X_tr, X_val, y_tr, y_val = train_test_split(
    X, y_cat, test_size=0.1, random_state=42, stratify=y
)

# Аугментация для улучшения качества предсказания
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X_tr)

# Колбэки для мониторинга
callbacks = [
    ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6),
    ModelCheckpoint('best_fmnist.h5', monitor='val_accuracy', save_best_only=True),
    EarlyStopping(monitor='val_accuracy', patience=7, restore_best_weights=True)
]

In [ ]:
# 7. Тренировка
history = model.fit(
    datagen.flow(X_tr, y_tr, batch_size=128),
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=callbacks
)

In [ ]:
# 8. Предсказания на тесте и создание файла для сабмита
preds = model.predict(X_test)
labels = preds.argmax(axis=1)

submission = pd.DataFrame({
    'Id': test['Id'],      # или корректное имя колонки
    'label': labels
})
submission.to_csv('submission.csv', index=False)
print("Saved submission.csv")
